In [6]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

In [7]:
data = pd.read_csv('DeltaMF_data.csv')
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,...,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price_Change
0,SAM,2013-12-01,84.6,27.6,57.0,0.0,4.40,54.6,4.21,0.0,...,11.5,10.9,4.7,-36.0,-18.6,-49.9,-38.9,-38.9,-2.43,107.34
1,CMS,2019-12-01,2308.0,2045.0,263.0,0.5,0.90,263.0,0.90,1516.0,...,24.3,23.0,87.0,-210.0,134.0,11.0,114.0,152.7,0.20,13.19
2,HON,2019-12-01,906.0,558.0,348.0,-16.5,1.09,702.0,0.93,493.0,...,-179.0,-622.0,463.0,-1560.0,-1568.0,-2448.0,-816.5,-822.8,0.85,44.88
3,FFIV,2011-09-01,206.3,104.6,101.7,-1.4,1.48,99.6,1.40,0.0,...,75.1,90.2,103.3,98.5,-209.9,-9.9,71.8,71.8,0.94,-32.76
4,CCL,2018-11-01,1623.0,1396.0,227.0,-22.0,1.39,293.0,1.31,1120.0,...,91.9,546.0,227.0,-392.0,992.0,815.0,-906.3,-908.7,-0.75,-5.35
5,NEE,2017-12-01,7970.0,3770.0,4200.0,3.2,7.94,3876.0,7.92,4185.0,...,1003.8,2474.0,89.0,-872.0,464.0,-303.0,-1651.4,-1363.8,-2.08,36.73
6,DOV,2014-12-01,-1824.9,-148.2,-1676.7,-7.0,-8.95,-2170.3,-13.06,202.5,...,56.6,-227.9,-176.7,-333.4,423.0,-126.1,3940.9,3945.4,-1.04,-6.75
7,AGI,2019-12-01,124.3,31.3,93.0,2.3,0.24,93.0,0.24,-2.2,...,96.2,168.7,46.5,-71.0,-8.2,-28.4,-29.9,-30.3,0.01,2.42
8,RL,2017-04-01,-561.1,-117.2,-443.9,-1.9,-4.42,-406.5,-4.29,-161.4,...,-61.4,-495.7,-64.1,368.2,-35.4,245.1,130.7,125.4,1.05,-12.49
9,IP,2013-12-01,-625.0,-2436.0,1811.0,-2.4,4.24,2129.0,4.92,-652.0,...,83.9,601.0,61.0,4031.0,-893.0,3192.0,1297.1,1251.5,0.49,9.06


In [8]:
prices = data['Price_Change'] 
data = data.drop(['Company', 'Time', 'Price_Change'], axis = 1)
data

,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Total Revenues,...,EBIT,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share
0,84.6,27.6,57.0,0.0,4.40,54.6,4.21,0.0,24.9,158.9,...,19.0,11.5,10.9,4.7,-36.0,-18.6,-49.9,-38.9,-38.9,-2.43
1,2308.0,2045.0,263.0,0.5,0.90,263.0,0.90,1516.0,1529.0,-28.0,...,78.0,24.3,23.0,87.0,-210.0,134.0,11.0,114.0,152.7,0.20
2,906.0,558.0,348.0,-16.5,1.09,702.0,0.93,493.0,987.0,-5093.0,...,-398.0,-179.0,-622.0,463.0,-1560.0,-1568.0,-2448.0,-816.5,-822.8,0.85
3,206.3,104.6,101.7,-1.4,1.48,99.6,1.40,0.0,-114.1,269.8,...,120.7,75.1,90.2,103.3,98.5,-209.9,-9.9,71.8,71.8,0.94
4,1623.0,1396.0,227.0,-22.0,1.39,293.0,1.31,1120.0,533.0,1371.0,...,146.0,91.9,546.0,227.0,-392.0,992.0,815.0,-906.3,-908.7,-0.75
5,7970.0,3770.0,4200.0,3.2,7.94,3876.0,7.92,4185.0,3763.0,1035.0,...,1685.0,1003.8,2474.0,89.0,-872.0,464.0,-303.0,-1651.4,-1363.8,-2.08
6,-1824.9,-148.2,-1676.7,-7.0,-8.95,-2170.3,-13.06,202.5,324.8,597.6,...,93.9,56.6,-227.9,-176.7,-333.4,423.0,-126.1,3940.9,3945.4,-1.04
7,124.3,31.3,93.0,2.3,0.24,93.0,0.24,-2.2,6.0,31.3,...,148.6,96.2,168.7,46.5,-71.0,-8.2,-28.4,-29.9,-30.3,0.01
8,-561.1,-117.2,-443.9,-1.9,-4.42,-406.5,-4.29,-161.4,-428.7,-752.4,...,-116.6,-61.4,-495.7,-64.1,368.2,-35.4,245.1,130.7,125.4,1.05
9,-625.0,-2436.0,1811.0,-2.4,4.24,2129.0,4.92,-652.0,-1152.0,1631.0,...,140.0,83.9,601.0,61.0,4031.0,-893.0,3192.0,1297.1,1251.5,0.49


In [9]:
col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price_Change', prices)
data


Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done
Total Revenues
Done
Operating Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done
Net Income
Done
Cash from Operations
Done
Cash from Investing
Done
Cash from Financing
Done
Net Change in Cash
Done
Levered Free Cash Flow
Done
Unlevered Free Cash Flow
Done
Free Cash Flow / Share
Done


,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Total Revenues,...,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price_Change
0,0.558733,0.667560,0.518349,0.206256,0.001096,0.488254,0.156606,0.649890,0.600700,0.551459,...,0.230997,0.480983,0.465090,0.503471,0.538870,0.605671,0.540246,0.545010,0.437558,107.34
1,0.565675,0.675479,0.519654,0.206277,0.001025,0.489663,0.156548,0.660569,0.610258,0.550661,...,0.231157,0.481047,0.467133,0.502132,0.539847,0.606066,0.540994,0.545940,0.437728,13.19
2,0.561298,0.669642,0.520193,0.205553,0.001029,0.492632,0.156549,0.653363,0.606814,0.529027,...,0.228619,0.477635,0.476467,0.491749,0.528946,0.590106,0.536446,0.541208,0.437770,44.88
3,0.559113,0.667862,0.518632,0.206196,0.001037,0.488558,0.156557,0.649890,0.599816,0.551933,...,0.231791,0.481402,0.467538,0.504505,0.537644,0.605930,0.540788,0.545547,0.437776,-32.76
4,0.563536,0.672931,0.519426,0.205319,0.001035,0.489866,0.156556,0.657780,0.603929,0.556636,...,0.232001,0.483813,0.470609,0.500733,0.545342,0.611285,0.536007,0.540791,0.437666,-5.35
5,0.583353,0.682250,0.544603,0.206392,0.001169,0.514095,0.156672,0.679369,0.624456,0.555201,...,0.243385,0.494010,0.467183,0.497041,0.541960,0.604028,0.532365,0.538584,0.437580,36.73
6,0.552771,0.666870,0.507362,0.205958,0.000824,0.473209,0.156303,0.651317,0.602606,0.553333,...,0.231560,0.479719,0.460587,0.501183,0.541698,0.605176,0.559698,0.564336,0.437648,-6.75
7,0.558857,0.667575,0.518577,0.206354,0.001011,0.488514,0.156537,0.649875,0.600580,0.550914,...,0.232055,0.481817,0.466128,0.503201,0.538936,0.605810,0.540290,0.545052,0.437715,2.42
8,0.556717,0.666992,0.515175,0.206175,0.000916,0.485136,0.156457,0.648754,0.597817,0.547567,...,0.230087,0.478303,0.463382,0.506579,0.538762,0.607586,0.541075,0.545807,0.437783,-12.49
9,0.556518,0.657890,0.529464,0.206154,0.001093,0.502281,0.156619,0.645298,0.593220,0.557747,...,0.231901,0.484104,0.466488,0.534751,0.533269,0.626713,0.546776,0.551269,0.437746,9.06


In [10]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:26] # Parameters
y = NpMatrix[:,26] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.5587332  0.66756006 0.51834883 ... 0.54024647 0.54501042 0.43755785]
 [0.56567503 0.67547888 0.51965425 ... 0.54099377 0.54593977 0.43772771]
 [0.56129775 0.66964202 0.5201929  ... 0.53644599 0.54120814 0.4377697 ]
 ...
 [0.52626043 0.63937353 0.49794427 ... 0.57679926 0.58110907 0.44130202]
 [0.56089124 0.66865403 0.52096285 ... 0.53194171 0.53674037 0.43735956]
 [0.55865951 0.66917098 0.51559856 ... 0.54612999 0.55081934 0.43800221]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 26
Total Number of Training instances: 4500


[107.34  13.19  44.88 ... -28.45  57.36   9.56]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 107.34
Total number of y values 4500


In [11]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=26, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))
    
    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=13, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))


    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='tanh'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [12]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 500, 
                                 batch_size=5)

In [13]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 3600 samples
Epoch 1/500
3600/3600 [==============================] - 2s 522us/sample - loss: 15.9645 - mae: 15.9645 - mse: 2784.0073
Epoch 2/500
3600/3600 [==============================] - 1s 264us/sample - loss: 15.9389 - mae: 15.9389 - mse: 2782.1711
Epoch 3/500
3600/3600 [==============================] - 1s 379us/sample - loss: 15.9380 - mae: 15.9380 - mse: 2782.0898
Epoch 4/500
3600/3600 [==============================] - 1s 339us/sample - loss: 15.9373 - mae: 15.9373 - mse: 2782.0762
Epoch 5/500
3600/3600 [==============================] - 1s 268us/sample - loss: 15.9370 - mae: 15.9370 - mse: 2782.0737
Epoch 6/500
3600/3600 [==============================] - 1s 232us/sample - loss: 15.9374 - mae: 15.9374 - mse: 2782.0718
Epoch 7/500
3600/3600 [==============================] - 1s 251us/sample - loss: 15.9372 - mae: 15.9372 - mse: 2782.0740
Epoch 8/500
3600/3600 [==============================] - 1s 263us/sample - loss: 15.9373 - mae: 15.9373 - mse: 2782.0742
Epoch 9/50

3600/3600 [==============================] - 2s 525us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0688 - loss: 14.1763 - ma
Epoch 68/500
3600/3600 [==============================] - 2s 589us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0681
Epoch 69/500
3600/3600 [==============================] - 1s 319us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0688
Epoch 70/500
3600/3600 [==============================] - 1s 303us/sample - loss: 15.9372 - mae: 15.9372 - mse: 2782.0706
Epoch 71/500
3600/3600 [==============================] - 1s 377us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0698
Epoch 72/500
3600/3600 [==============================] - 2s 453us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0684
Epoch 73/500
3600/3600 [==============================] - 2s 450us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0703
Epoch 74/500
3600/3600 [==============================] - 2s 421us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0679
Epoch 75/500
360

3600/3600 [==============================] - 1s 392us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0681
Epoch 134/500
3600/3600 [==============================] - 1s 388us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0684
Epoch 135/500
3600/3600 [==============================] - 2s 443us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0674
Epoch 136/500
3600/3600 [==============================] - 1s 404us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0674
Epoch 137/500
3600/3600 [==============================] - 1s 397us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0684
Epoch 138/500
3600/3600 [==============================] - 2s 540us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0664
Epoch 139/500
3600/3600 [==============================] - 2s 428us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0691
Epoch 140/500
3600/3600 [==============================] - 1s 392us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0681
Epoch 141/500
3600/3600 [=====

3600/3600 [==============================] - 2s 446us/sample - loss: 15.9374 - mae: 15.9374 - mse: 2782.0698
Epoch 200/500
3600/3600 [==============================] - 1s 413us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0691
Epoch 201/500
3600/3600 [==============================] - 2s 427us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0671
Epoch 202/500
3600/3600 [==============================] - 2s 430us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0693
Epoch 203/500
3600/3600 [==============================] - 2s 508us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0667
Epoch 204/500
3600/3600 [==============================] - 1s 398us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0664
Epoch 205/500
3600/3600 [==============================] - 2s 492us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0688
Epoch 206/500
3600/3600 [==============================] - 2s 478us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0679
Epoch 207/500
3600/3600 [=====

3600/3600 [==============================] - 1s 389us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0662
Epoch 266/500
3600/3600 [==============================] - 1s 381us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0684
Epoch 267/500
3600/3600 [==============================] - 1s 383us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0671
Epoch 268/500
3600/3600 [==============================] - 2s 425us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0688
Epoch 269/500
3600/3600 [==============================] - 1s 392us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0706
Epoch 270/500
3600/3600 [==============================] - 1s 375us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0691
Epoch 271/500
3600/3600 [==============================] - 1s 403us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0671
Epoch 272/500
3600/3600 [==============================] - 2s 437us/sample - loss: 15.9371 - mae: 15.9371 - mse: 2782.0706
Epoch 273/500
3600/3600 [=====

...

3600/3600 [==============================] - 2s 420us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1174
Epoch 202/500
3600/3600 [==============================] - 2s 445us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1187
Epoch 203/500
3600/3600 [==============================] - 1s 399us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1194
Epoch 204/500
3600/3600 [==============================] - 1s 396us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1177
Epoch 205/500
3600/3600 [==============================] - 2s 452us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1174
Epoch 206/500
3600/3600 [==============================] - 1s 400us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1169
Epoch 207/500
3600/3600 [==============================] - 2s 419us/sample - loss: 15.9075 - mae: 15.9075 - mse: 2490.1189
Epoch 208/500
3600/3600 [==============================] - 2s 428us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1182
Epoch 209/500
3600/3600 [=====

3600/3600 [==============================] - 2s 449us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1172
Epoch 268/500
3600/3600 [==============================] - 1s 396us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1184
Epoch 269/500
3600/3600 [==============================] - 1s 403us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1155
Epoch 270/500
3600/3600 [==============================] - 1s 391us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1169
Epoch 271/500
3600/3600 [==============================] - 2s 451us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1187
Epoch 272/500
3600/3600 [==============================] - 1s 406us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1191
Epoch 273/500
3600/3600 [==============================] - 1s 403us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1177
Epoch 274/500
3600/3600 [==============================] - 1s 403us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1169
Epoch 275/500
3600/3600 [=====

3600/3600 [==============================] - 1s 399us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1194
Epoch 334/500
3600/3600 [==============================] - 2s 443us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1172
Epoch 335/500
3600/3600 [==============================] - 1s 387us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1182
Epoch 336/500
3600/3600 [==============================] - 1s 408us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1184
Epoch 337/500
3600/3600 [==============================] - 1s 394us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1204
Epoch 338/500
3600/3600 [==============================] - 2s 436us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1196
Epoch 339/500
3600/3600 [==============================] - 1s 404us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1167
Epoch 340/500
3600/3600 [==============================] - 1s 412us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1182
Epoch 341/500
3600/3600 [=====

3600/3600 [==============================] - 2s 426us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1191
Epoch 400/500
3600/3600 [==============================] - 2s 518us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1184
Epoch 401/500
3600/3600 [==============================] - 2s 438us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1187
Epoch 402/500
3600/3600 [==============================] - 1s 379us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1177
Epoch 403/500
3600/3600 [==============================] - 1s 378us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1194
Epoch 404/500
3600/3600 [==============================] - 2s 423us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1194
Epoch 405/500
3600/3600 [==============================] - 1s 380us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1182
Epoch 406/500
3600/3600 [==============================] - 1s 379us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1196
Epoch 407/500
3600/3600 [=====

3600/3600 [==============================] - 2s 430us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1194
Epoch 466/500
3600/3600 [==============================] - 1s 381us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1189
Epoch 467/500
3600/3600 [==============================] - 1s 384us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1177
Epoch 468/500
3600/3600 [==============================] - 1s 379us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1194
Epoch 469/500
3600/3600 [==============================] - 2s 418us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1201
Epoch 470/500
3600/3600 [==============================] - 1s 374us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1189
Epoch 471/500
3600/3600 [==============================] - 1s 377us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1189
Epoch 472/500
3600/3600 [==============================] - 1s 373us/sample - loss: 15.9077 - mae: 15.9077 - mse: 2490.1194
Epoch 473/500
3600/3600 [=====

In [14]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-15.77494436 -15.54371109 -13.65947783 -18.65256681 -15.89251099]
Mean MAE: -15.904642215569814
Standard Deviation of the mean: 1.5961887506372878
